In [1]:
import codecs
import requests
import re
import datetime
import threading
import time
from bs4 import BeautifulSoup

import sys
reload(sys)
sys.setdefaultencoding('utf-8')

class Ilbe(object):
    def __init__(self, date_from, date_to, keyword, start_page):
        self.date_from = date_from
        self.date_to = date_to
        self.keyword = keyword
        self.start_page = start_page

    def str_process(self, string):
        #del_blank = re.compile(r'(\s+)')
        del_escape = re.compile(r'(\n)(|\t)')
        #string = del_blank.sub(' ', string)
        string = del_escape.sub('', string)
        return string
        
    # on Board
    def crawl(self):
        f = codecs.open('ilbe_{}.txt'.format(self.keyword), 'a', 'utf-8')
        page_num = self.start_page
        #view_flag = -1
        num_contents = 0
        
        while True:
            hdr = {'User-Agent' : 'Mozilla/5.0'}
            url = "http://www.ilbe.com/index.php?mid=smartphone&search_target=title_content&search_keyword={0}&page={1}".format(self.keyword, self.start_page)
            res = requests.get(url, headers = hdr)
            soup = BeautifulSoup(res.content, 'lxml')
            links = soup.find_all('td', attrs={'class': 'title'})

            date_soup = soup.find_all('td', attrs={'class': 'date'})
            
            try:
                if ':' in date_soup[3].get_text():
                    chk_date = datetime.datetime.today().strftime('%Y%m')
                else:
                    raw_date = re.search(r'\d{4}-\d{2}', date_soup[3].get_text()).group()
                    chk_date = raw_date.replace('-', '')
            except IndexError:
                pass
                    
            print(self.start_page)
            self.start_page += 1
            #view_flag += 1
            threads = []

            if int(chk_date) <= int(self.date_to) and int(chk_date) > int(self.date_from):
                for link in links:
                    try:
                        link = link.a['href']
                        res2 = requests.get(link, headers = hdr)
                        soup2 = BeautifulSoup(res2.content, 'lxml')
                        contents = soup2.find_all('div', attrs={'class': 'originalContent'})
                        
                        if link[-9:] == '214481493' or link[-10:] == '1877136960':
                            pass
                        
                        else:
                            #view_index = num_contents - view_flag * 22
                            num_contents += 1
                            print('processing... ' + str(num_contents) + ' ' + str(link))
                            try:
                                date = self.str_process(contents[0].select('div.date')[0].get_text()[0:11].replace('.', '-'))
                            except IndexError:
                                pass
                            try:
                                #view = self.str_process(soup2.select('td.reading')[view_index].get_text())
                                #print(num_contents,'-',view_flag,'*',view_index)
                                title = self.str_process(contents[0].select('div.title')[0].get_text())
                                content = self.str_process(contents[0].select('div#copy_layer_1')[0].get_text())
                                c_time = datetime.datetime.now().strftime('%Y-%m-%d %H:%M')

                                f.write(link+'\t'+date+'\t'+title+'\t'+content+'\t'+c_time+'\n')
                            except IndexError:
                                pass
                    except TypeError:
                        pass
                    
            elif int(chk_date) == int(self.date_from):
                print('Reached to date limit ('+str(self.date_to)+', yyyy-mm)')
                break
            else:
                num_contents += 1
                pass
        f.close()
        print("Contents crawling has been completed, total " + str(num_contents) + ' contents')

In [3]:
i_g5 = Ilbe(201512, 201607, 'g5', 1)
i_g5.crawl()

#i_g6 = Ilbe(201612, 201703, 'g6', 46)
#i_g6.crawl()

#i_s6 = Ilbe(201412, 201507, 's6', 1)
#i_s6.crawl()

#i_s7 = Ilbe(201512, 201607, 's7', 455)
#i_s7.crawl()

In [4]:
f.close()

NameError: name 'f' is not defined

In [ ]:
link = "http://www.ilbe.com/index.php?mid=smartphone&search_target=title_content&search_keyword=g5&page=10&document_srl=9378278766"
res2 = requests.get(link)
soup2 = BeautifulSoup(res2.content, 'lxml')
contents = soup2.find_all('div', attrs={'class': 'originalContent'})

In [ ]:
def str_process(string):
    #del_blank = re.compile(r'(\s+)')
    del_escape = re.compile(r'(\n)(|\t)')
    #string = del_blank.sub(' ', string)
    string = del_escape.sub('', string)
    return string

date = str_process(contents[0].select('div.date')[0].get_text()[0:11].replace('.', '-'))
title = str_process(contents[0].select('div.title')[0].get_text())
content = str_process(contents[0].select('div#copy_layer_1')[0].get_text())

print(date)

In [14]:
import codecs
import requests
import re
import datetime
from threading import Thread
from threading import Lock
from Queue import Queue
from bs4 import BeautifulSoup

import sys
reload(sys)
sys.setdefaultencoding('utf-8')

class Ilbe(object):
    def __init__(self, date_from, date_to, keyword, start_page):
        self.date_from = date_from
        self.date_to = date_to
        self.keyword = keyword
        self.start_page = start_page
        
        self.hdr = {'User-Agent' : 'Mozilla/5.0'}
        self.view_flag = 0
        self.num_contents = 0
        
        self.lock = Lock()
        self.n = 0
        
        self.f = codecs.open('ilbe_{}.txt'.format(self.keyword), 'a', 'utf-8')

    def str_process(self, string):
        del_escape = re.compile(r'(\n)(|\t)')
        string = del_escape.sub('', string)
        return string
    
    def get_contents(self, link):
        link = link
        print(link)
        res = requests.get(link, headers = self.hdr)
        soup = BeautifulSoup(res.content, 'lxml')
        contents = soup.find_all('div', attrs={'class': 'originalContent'})

        if link[-9:] == '214481493' or link[-10:] == '1877136960':
            pass

        else:
            view_index = self.num_contents - self.view_flag * 22
            self.num_contents += 1
            #print('processing... ' + str(self.num_contents) + ' ' + str(link))
            print(contents[0].select('div.date')[0].get_text()[0:11].replace('.', '-'))

            #date = self.str_process(contents[0].select('div.date')[0].get_text()[0:11].replace('.', '-'))
            #view = self.str_process(soup.select('td.reading')[view_index].get_text())
            #title = self.str_process(contents[0].select('div.title')[0].get_text())
            #content = self.str_process(contents[0].select('div#copy_layer_1')[0].get_text())
            #time = datetime.datetime.now().strftime('%Y-%m-%d %H:%M')

            #self.f.write(link+'\t'+date+'\t'+view+'\t'+title+'\t'+content+'\t'+time+'\n')

    # on Board
    def crawl(self):
        while True:
            url = "http://www.ilbe.com/index.php?mid=smartphone&search_target=title_content&search_keyword={0}&page={1}".format(self.keyword, self.start_page)
            res = requests.get(url, headers = self.hdr)
            soup = BeautifulSoup(res.content, 'lxml')
            links = soup.find_all('td', attrs={'class': 'title'})

            date_soup = soup.find_all('td', attrs={'class': 'date'})
            
            if ':' in date_soup[2].get_text():
                chk_date = datetime.datetime.today().strftime('%Y%m')
            else:
                raw_date = re.search(r'\d{4}-\d{2}', date_soup[3].get_text()).group()
                chk_date = raw_date.replace('-', '')

            print(self.start_page)
            self.start_page += 1
            self.view_flag += 1
            
            threads = []

            if int(chk_date) <= int(self.date_to) and int(chk_date) > int(self.date_from):
                for link in links:
                    link = link.a['href']
                    
                    t = threading.Thread(target=self.get_contents, args=(link,))

                    with self.lock:
                        self.n += 1
                        
                    t.start()
                    
                for t in threads:
                    t.join()
                    
            elif int(chk_date) == int(self.date_from):
                print('Reached to date limit ('+str(self.date_to)+', yyyy-mm)')
                break
            else:
                pass
        self.f.close()
        print("Contents crawling has been completed, total " + str(self.num_contents) + ' contents')
        

In [15]:
i_g5 = Ilbe(201702, 201703, 'g5', 4)
i_g5.crawl()

IndexError: list index out of range

In [7]:
url = "http://www.ilbe.com/index.php?mid=smartphone&search_target=title_content&search_keyword=s7&page=455&document_srl=7855756735"
res = requests.get(url)
soup = BeautifulSoup(res.content, 'lxml')
links = soup.find_all('td', attrs={'class': 'title'})

date_soup = soup.find_all('td', attrs={'class': 'date'})
date_soup[2].get_text()

u'2016-04-10'